<a href="https://colab.research.google.com/github/thinothw/DFDS-Final-Project/blob/main/Phase01_Sandbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# INFRASTRUCTURE - Connect to Drive

from google.colab import drive
import os

print("Requesting Google Drive access...")
drive.mount('/content/drive')

# Double check the project folder exists
base_path = '/content/drive/MyDrive/Deepfake_Honours_Project/Sandbox_10K'
if os.path.exists(base_path):
    print(f"✅ Connection Stable! Project folder found at: {base_path}")
else:
    print(f"⚠️ Warning: Base path not found. Check your Drive folder name!")

Requesting Google Drive access...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Connection Stable! Project folder found at: /content/drive/MyDrive/Deepfake_Honours_Project/Sandbox_10K


In [1]:
# Clean, compatible install block
!pip uninstall -y numpy -q
!pip install -q numpy==1.26.4
!pip install -q datasets==2.18.0
!pip install -q facenet-pytorch==2.5.3
!pip install -q opencv-python==4.8.0.76

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 85.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-contrib-python 4.13.0.92 requires numpy>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.13.0.92 requires numpy>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
tobler 0.13.0 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.13.0.92 requires numpy>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1

In [1]:
!pip install retina-face opencv-python

In [1]:
import numpy
import pandas
import torch
print("✅ All core imports successful. The environment is stable.")

✅ All core imports successful. The environment is stable.


In [3]:

# 1. Mount Google Drive

from google.colab import drive
import os
from datasets import load_dataset
import torch

print("Requesting Google Drive access...")
drive.mount('/content/drive')

# Verify Drive mounted correctly
if not os.path.exists('/content/drive/MyDrive'):
    raise RuntimeError("Google Drive mount failed.")
print("Drive mounted successfully.")


# 2. GPU Verification

print("\nChecking GPU availability...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device detected: {device}")


# 3. Create Clean Folder Structure

base_path = '/content/drive/MyDrive/Deepfake_Honours_Project/Sandbox_10K'
raw_data_path = os.path.join(base_path, 'raw_data')

real_path = os.path.join(raw_data_path, 'real')
fake_path = os.path.join(raw_data_path, 'fake')

os.makedirs(real_path, exist_ok=True)
os.makedirs(fake_path, exist_ok=True)

print("Folder structure verified.")


# 4. Connect to OpenFake Dataset

print("\nConnecting to OpenFake dataset stream...")
dataset_name = "ComplexDataLab/OpenFake"

try:
    openfake = load_dataset(dataset_name, split='train', streaming=True)
    print("Dataset connection successful.")
except Exception as e:
    raise RuntimeError(f"Dataset loading failed: {e}")


# 5. Dataset Schema & Label Sanity Check

print("\nPerforming schema + label sanity check...")

sample_preview = None
for i, sample in enumerate(openfake):
    sample_preview = sample
    break

if sample_preview is None:
    raise RuntimeError("Dataset appears empty.")

print("Sample keys:", sample_preview.keys())

if 'label' not in sample_preview or 'image' not in sample_preview:
    raise RuntimeError("Dataset schema unexpected. Required fields missing.")

print("Label of first sample:", sample_preview['label'])

# Reset stream after preview
openfake = load_dataset(dataset_name, split='train', streaming=True)


# 6. Download 25 Real + 25 Fake Images

real_count = 0
fake_count = 0
save_errors = 0
max_iterations = 10000
iteration_counter = 0

print("\nStarting micro-download (25 Real + 25 Fake)...")

for item in openfake:
    iteration_counter += 1

    if iteration_counter > max_iterations:
        raise RuntimeError("Iteration cap exceeded. Possible label imbalance or streaming issue.")

    try:
        label = item['label']
        image = item['image']

        # THE FIX - Looking for strings instead of integers
        if label not in ['real', 'fake']:
            continue  # skip unexpected labels safely

        if label == 'real' and real_count < 25:
            image.save(os.path.join(real_path, f"test_openfake_real_{real_count}.jpg"))
            real_count += 1

        elif label == 'fake' and fake_count < 25:
            image.save(os.path.join(fake_path, f"test_openfake_fake_{fake_count}.jpg"))
            fake_count += 1

        if real_count == 25 and fake_count == 25:
            break

    except Exception as e:
        save_errors += 1
        print(f"Error saving image: {e}")


# 7. Final Integrity Check

print("\nDownload Summary:")
print(f"Iterations used: {iteration_counter}")
print(f"Real images saved: {real_count}")
print(f"Fake images saved: {fake_count}")
print(f"Save errors: {save_errors}")

if real_count != 25 or fake_count != 25:
    raise RuntimeError("Image count mismatch. Micro-test failed.")

print("\n✅ Micro-test completed successfully and verified.")

Requesting Google Drive access...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive mounted successfully.

Checking GPU availability...
Device detected: cpu
Folder structure verified.

Connecting to OpenFake dataset stream...


Resolving data files:   0%|          | 0/206 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/206 [00:00<?, ?it/s]

Dataset connection successful.

Performing schema + label sanity check...
Sample keys: dict_keys(['image', 'prompt', 'label', 'model', 'type', 'release_date'])
Label of first sample: real


Resolving data files:   0%|          | 0/206 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/206 [00:00<?, ?it/s]


Starting micro-download (25 Real + 25 Fake)...

Download Summary:
Iterations used: 61
Real images saved: 25
Fake images saved: 25
Save errors: 0

✅ Micro-test completed successfully and verified.


In [5]:
# PHASE 2 - RetinaFace V3

import os
import cv2
import numpy as np
from retinaface import RetinaFace
from PIL import Image
import warnings

warnings.filterwarnings("ignore")

print("Bouncer operating on: CPU (RetinaFace V4.3)\n")

# Paths
raw_real = os.path.join(base_path, 'raw_data/real')
raw_fake = os.path.join(base_path, 'raw_data/fake')

hq_real = os.path.join(base_path, 'processed_data_V4_3/real')
hq_fake = os.path.join(base_path, 'processed_data_V4_3/fake')

reject_real = os.path.join(base_path, 'processed_data_V4_3/rejected/real')
reject_fake = os.path.join(base_path, 'processed_data_V4_3/rejected/fake')

for path in [hq_real, hq_fake, reject_real, reject_fake]:
    os.makedirs(path, exist_ok=True)

# Blur Detection
def blur_score(pil_image):
    img = np.array(pil_image)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    return cv2.Laplacian(gray, cv2.CV_64F).var()

# RetinaFace Extraction Engine (Pragmatic Filters)
def hq_crop_engine_v4_3(
    img_path,
    save_path,
    reject_path,
    padding=25,
    min_face_size=30,      #  Lowered to allow gritty/low-res data
    min_face_ratio=0.005,  #  Lowered to 0.5% to allow wider shots
    blur_threshold=50,
    confidence_threshold=0.90
):
    try:
        faces = RetinaFace.detect_faces(img_path)

        if not isinstance(faces, dict) or len(faces) == 0:
            return False, "No Face Detected"

        largest_area = 0
        best_face = None

        for face in faces.values():
            box = face.get('facial_area', None)
            if box is None:
                continue

            area = (box[2] - box[0]) * (box[3] - box[1])

            if area > largest_area:
                largest_area = area
                best_face = face

        if best_face is None:
            return False, "No Valid Face Found"

        box = best_face['facial_area']
        confidence = best_face['score']

        if confidence < confidence_threshold:
            return False, f"Low Confidence ({confidence:.2f})"

        img_pil = Image.open(img_path).convert('RGB')
        width, height = img_pil.size

        face_w = box[2] - box[0]
        face_h = box[3] - box[1]
        face_area = face_w * face_h
        image_area = width * height

        if face_w < min_face_size or face_h < min_face_size:
            return False, f"Too Small Pixels ({int(face_w)}x{int(face_h)})"

        ratio = face_area / image_area
        if ratio < min_face_ratio:
            return False, f"Too Small Ratio ({ratio:.3f})"

        x_min = max(0, int(box[0]) - padding)
        y_min = max(0, int(box[1]) - padding)
        x_max = min(width, int(box[2]) + padding)
        y_max = min(height, int(box[3]) + padding)

        cropped_img = img_pil.crop((x_min, y_min, x_max, y_max))

        blur_val = blur_score(cropped_img)
        if blur_val < blur_threshold:
            return False, f"Blurred ({blur_val:.2f})"

        cropped_img.save(save_path)
        return True, "Success"

    except Exception as e:
        return False, str(e)

# Dataset Cleaner
def clean_dataset_v4_3(input_folder, output_folder, reject_folder, label):
    print(f"\n--- Cleaning {label} Folder ---")
    images = [f for f in os.listdir(input_folder) if f.endswith('.jpg')]

    success = 0
    rejected = 0
    reasons = {}

    for name in images:
        img_path = os.path.join(input_folder, name)
        save_path = os.path.join(output_folder, name)
        reject_path = os.path.join(reject_folder, name)

        ok, msg = hq_crop_engine_v4_3(img_path, save_path, reject_path)

        if ok:
            success += 1
            print(f"  [✓] EXTRACTED: {name}")
        else:
            rejected += 1
            print(f"  [X] REJECTED: {name} -> Reason: {msg}")
            reasons[msg] = reasons.get(msg, 0) + 1
            Image.open(img_path).save(reject_path)

    print(f"\n✅ FINAL STATS for {label}: {success} Valid | {rejected} Rejected")
    print("Rejection Breakdown:")
    for k, v in reasons.items():
        print(f"   {k} → {v}")
    print("="*40)

# Run
clean_dataset_v4_3(raw_real, hq_real, reject_real, "REAL")
clean_dataset_v4_3(raw_fake, hq_fake, reject_fake, "FAKE")

print("\n Phase 2.8 Complete! Check processed_data_V4_3")

Bouncer operating on: CPU (RetinaFace V4.3)


--- Cleaning REAL Folder ---
  [✓] EXTRACTED: test_openfake_real_0.jpg
  [✓] EXTRACTED: test_openfake_real_1.jpg
  [X] REJECTED: test_openfake_real_2.jpg -> Reason: Too Small Ratio (0.005)
  [✓] EXTRACTED: test_openfake_real_3.jpg
  [X] REJECTED: test_openfake_real_4.jpg -> Reason: No Face Detected
  [✓] EXTRACTED: test_openfake_real_5.jpg
  [✓] EXTRACTED: test_openfake_real_6.jpg
  [X] REJECTED: test_openfake_real_7.jpg -> Reason: No Face Detected
  [✓] EXTRACTED: test_openfake_real_8.jpg
  [X] REJECTED: test_openfake_real_9.jpg -> Reason: Too Small Ratio (0.002)
  [X] REJECTED: test_openfake_real_10.jpg -> Reason: Too Small Pixels (24x32)
  [X] REJECTED: test_openfake_real_11.jpg -> Reason: Too Small Pixels (24x24)
  [✓] EXTRACTED: test_openfake_real_12.jpg
  [✓] EXTRACTED: test_openfake_real_13.jpg
  [✓] EXTRACTED: test_openfake_real_14.jpg
  [✓] EXTRACTED: test_openfake_real_15.jpg
  [✓] EXTRACTED: test_openfake_real_16.jpg
  [✓] EXTRAC